In [2]:
import tensorflow as tf

# Emotion Mapping (Negative = 0, Positive = 1)
emotion_mapping = {
    "angry": 0, "contempt": 0, "disgust": 0, "fear": 0, "sad": 0,
    "happy": 1, "neutral": 1, "surprise": 1
}

# Function to map labels correctly
def map_labels(image, label):
    class_names = train_dataset.class_names  # Get class names
    mapped_label = tf.constant(emotion_mapping[class_names[label.numpy()]], dtype=tf.int64)
    return image, mapped_label

# Wrapper function for `tf.py_function`
def tf_map_labels(image, label):
    mapped_image, mapped_label = tf.py_function(func=map_labels, inp=[image, label], Tout=(tf.float32, tf.int64))
    mapped_image.set_shape((224, 224, 3))  # Ensure shape consistency
    mapped_label.set_shape(())  # Ensure label is a scalar
    return mapped_image, mapped_label

# Load the dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "D:/depression_detect/datasets/AffectNet/train",
    image_size=(224, 224),
    batch_size=42,
    shuffle=True
)

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "D:/depression_detect/datasets/AffectNet/val",
    image_size=(224, 224),
    batch_size=42,
    shuffle=False
)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "D:/depression_detect/datasets/AffectNet/test",
    image_size=(224, 224),
    batch_size=42,
    shuffle=False
)

# Apply the mapping function correctly
train_dataset = train_dataset.map(tf_map_labels)
val_dataset = val_dataset.map(tf_map_labels)
test_dataset = test_dataset.map(tf_map_labels)

# Prefetch to improve performance
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

print("✅ Datasets loaded and labels mapped correctly.")


Found 37553 files belonging to 8 classes.
Found 800 files belonging to 8 classes.
Found 3200 files belonging to 8 classes.
✅ Datasets loaded and labels mapped correctly.


In [6]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# ✅ Emotion Mapping (Negative = 0, Positive = 1)
emotion_mapping = {
    "angry": 0, "contempt": 0, "disgust": 0, "fear": 0, "sad": 0,
    "happy": 1, "neutral": 1, "surprise": 1
}

# ✅ Load dataset
def load_dataset(directory, batch_size=16, validation_split=0.9, subset="training"):
    return tf.keras.preprocessing.image_dataset_from_directory(
        directory,
        image_size=(224, 224),
        batch_size=batch_size,
        shuffle=True,
        validation_split=validation_split,
        subset=subset,
        seed=42  # for consistent split
    )

train_dataset = load_dataset("D:/depression_detect/datasets/AffectNet/train", subset="training")
val_dataset = load_dataset("D:/depression_detect/datasets/AffectNet/train", subset="validation")
test_dataset = load_dataset("D:/depression_detect/datasets/AffectNet/test")

# ✅ Function to normalize and map labels
def preprocess(image, label):
    image = tf.image.convert_image_dtype(image, tf.float32)
    mapped_label = tf.gather(list(emotion_mapping.values()), label)
    return image, tf.cast(mapped_label, tf.float32)

# ✅ Apply preprocessing
train_dataset = train_dataset.map(preprocess)
val_dataset = val_dataset.map(preprocess)
test_dataset = test_dataset.map(preprocess)

# ✅ Prefetch
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(AUTOTUNE)
val_dataset = val_dataset.prefetch(AUTOTUNE)
test_dataset = test_dataset.prefetch(AUTOTUNE)

print("✅ Datasets loaded and labels mapped correctly.")

# ✅ Load MobileNetV2 instead of ResNet50 for CPU efficiency
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# ✅ Freeze initial layers
base_model.trainable = False

# ✅ Build the model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(64, activation="relu"),
    layers.Dropout(0.4),
    layers.Dense(1, activation="sigmoid")
])

# ✅ Compile
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

# ✅ Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.5)

# ✅ Train (faster on CPU now)
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=20,
    callbacks=[early_stop, reduce_lr]
)

# ✅ Save model
model.save("D:/depression_detect/models/mobilenet_affectnet.keras", save_format="keras")

print("✅ Model trained and saved successfully.")


Found 37553 files belonging to 8 classes.
Using 3756 files for training.
Found 37553 files belonging to 8 classes.
Using 33797 files for validation.
Found 3200 files belonging to 8 classes.
Using 320 files for training.
✅ Datasets loaded and labels mapped correctly.
Epoch 1/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 294s 1s/step - accuracy: 0.5090 - loss: 0.7843 - val_accuracy: 0.5772 - val_loss: 0.6791 - learning_rate: 1.0000e-04
Epoch 2/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 248s 1s/step - accuracy: 0.5651 - loss: 0.6891 - val_accuracy: 0.5964 - val_loss: 0.6738 - learning_rate: 1.0000e-04
Epoch 3/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 248s 1s/step - accuracy: 0.5871 - loss: 0.6738 - val_accuracy: 0.5967 - val_loss: 0.6732 - learning_rate: 1.0000e-04
Epoch 4/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 249s 1s/step - accuracy: 0.5913 - loss: 0.6715 - val_accuracy: 0.5925 - val_loss: 0.6747 - learning_rate: 1.0000e-04
Epoch 5/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 249s 1s/step - accuracy: 0.5881 - loss: 0.6684 - val_accuracy: 0.

✅ Model trained and saved successfully.


In [7]:
test_loss, test_acc = model.evaluate(test_dataset)
print(f"✅ Test Accuracy: {test_acc:.4f} | Test Loss: {test_loss:.4f}")


20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 118ms/step - accuracy: 0.6829 - loss: 0.6237
✅ Test Accuracy: 0.6656 | Test Loss: 0.6341


In [8]:
import cv2
import numpy as np
import tensorflow as tf
import os

# ✅ Load Trained Model
model_path = "D:/depression_detect/models/mobilenet_affectnet.keras"
model = tf.keras.models.load_model(model_path)

# ✅ Define Emotion Mapping (Same as Training)
emotion_mapping = {
    "angry": 0, "contempt": 0, "disgust": 0, "fear": 0, "sad": 0,
    "happy": 1, "neutral": 1, "surprise": 1
}

# ✅ Load Video
video_path = "D:/depression_detect/videos/videoplayback.mp4"
cap = cv2.VideoCapture(video_path)

# ✅ Extract Frames & Predict
frame_interval = 5  # Process every 5th frame
predictions = []

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Stop if video ends

    if frame_count % frame_interval == 0:
        # Preprocess Frame
        resized_frame = cv2.resize(frame, (224, 224))
        normalized_frame = resized_frame / 255.0  # Normalize
        input_tensor = np.expand_dims(normalized_frame, axis=0)  # Add batch dim

        # Predict Emotion
        prediction = model.predict(input_tensor)[0][0]  # Get probability
        label = 1 if prediction > 0.5 else 0  # Convert to binary
        predictions.append(label)

    frame_count += 1

cap.release()
cv2.destroyAllWindows()

# ✅ Majority Vote: Final Facial Emotion Label
final_facial_label = 1 if np.mean(predictions) > 0.5 else 0
print(f"🟢 Final Facial Emotion: {'Positive' if final_facial_label else 'Negative'}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 917ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━

In [9]:
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model("D:/depression_detect/models/mobilenet_affectnet.keras")

# Now you can evaluate it
test_loss, test_acc = model.evaluate(test_dataset)
print(f"✅ Test Accuracy: {test_acc:.4f} | Test Loss: {test_loss:.4f}")


20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 129ms/step - accuracy: 0.6751 - loss: 0.6381
✅ Test Accuracy: 0.6656 | Test Loss: 0.6341


In [10]:
import cv2
import os

video_path = 'D:/depression_detection/videos/videoplayback.mp4'
output_dir = 'D:/depression_detection/preprocessed_data/frames/'
os.makedirs(output_dir, exist_ok=True)

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(video_path)
frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        face_resized = cv2.resize(face, (224, 224))
        cv2.imwrite(os.path.join(output_dir, f'frame_{frame_count}.jpg'), face_resized)
        break  # One face per frame is enough
    
    frame_count += 1

cap.release()
print("✅ Face frames extracted.")


✅ Face frames extracted.


In [12]:
import tensorflow as tf
import numpy as np
from keras.preprocessing import image
import os

model = tf.keras.models.load_model("D:\depression_detect\models\mobilenet_affectnet.keras")
frame_dir = "D:/depression_detection/preprocessed_data/frames/"

positive = 0
negative = 0

for frame in os.listdir(frame_dir):
    if not frame.endswith(".jpg"):
        continue
    img_path = os.path.join(frame_dir, frame)
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    prediction = model.predict(img_array)
    class_idx = np.argmax(prediction)

    if class_idx == 1:  # Assuming class 1 is Positive
        positive += 1
    else:
        negative += 1

# Decision logic
if positive > negative:
    print("🙂 Facial Expression: Positive")
elif negative > positive:
    print("😞 Facial Expression: Negative")
else:
    print("😐 Facial Expression: Neutral/Uncertain")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 845ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━